# Data Migration

In this notebook, we will perform a data migration from a CSV file to a PostgreSQL database. This process includes setting up the database connection, managing the table structure, and loading transformed data.

Here's what we will do:

1. Configure the Environment
2. Import the Modules
3. Connect to the Database
4. Create an SQLAlchemy Session
5. Create the Candidates Table in our Database
6. Transform and Load the Data


It's important to have the environment variables with your database credentials _(from the .env file)_ and the requirements _(installed via pip install -r requirements.txt)_.

---

First, we will load the environment variables from the .env file, which contains important configurations such as paths and credentials. Then, we will obtain the working directory from these variables and add it to the system path to ensure that the project's modules can be imported correctly.

In [8]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()
work_dir = os.getenv('WORK_DIR')

sys.path.append(work_dir)

print('Workdir: ', work_dir)

Workdir:  /Users/carol/Documents/workshop01


Now, we import the necessary modules and classes for the rest of the notebook.

In [9]:
from src.db_connection import build_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import inspect
from src.model import Candidates
from sqlalchemy.exc import SQLAlchemyError
from src.transform import Transform

The build_engine function is called to configure and create a connection to the PostgreSQL database.

In [10]:
engine = build_engine()

Successfully connected to the database workshop1!


A SQLAlchemy session is created using the database engine established in the previous step. Why? This session is necessary for performing read and write operations on the database.

In [11]:
Session = sessionmaker(bind=engine)
session = Session()

We check if the Candidates table already exists in the database. If it does, the table is dropped. Then, a new Candidates table is created. This ensures that the table is up-to-date and ready to receive new data. If any errors occur during this process, an error message is printed.

In [12]:
try:
    inspector = inspect(engine)

    if inspector.has_table('Candidates'):
        try:
            Candidates.__table__.drop(engine)
        except SQLAlchemyError as e:
            print(f"Error dropping table: {e}")
            raise
    try:
        Candidates.__table__.create(engine)
        print("Table creation was successful.")
    except SQLAlchemyError as e:
        print(f"Error creating table: {e}")
        raise

except SQLAlchemyError as error:
    print(f"An error occurred: {error}")

Table creation was successful.


An instance of the Transform class is created to read the file. Then, data transformations are applied, such as renaming columns and adding IDs. The transformed data is inserted into the Candidates table in the database. Finally, the database session is closed to free up resources.

In [13]:
try:
    df = Transform('../data/candidates.csv')
    
    df.rename_columns()
    df.insert_ids()
    
    df.df.to_sql('Candidates', engine, if_exists='append', index=False)
    
    print("Data uploaded successfully")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    if session:
        session.close()

Data uploaded successfully
